In [38]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("Missing").getOrCreate()

In [39]:
# Read the data
df = spark.read.csv(r"C:\Users\user\Desktop\test1.csv",header=True,inferSchema=True)
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [40]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [41]:
df.columns

# In pyspark we have to Grouped independent features [age,Experience] and convert that into a new column

['Name', 'age', 'Experience', 'Salary']

In [42]:
from pyspark.ml.feature import VectorAssembler
feature_assember = VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [43]:
output = feature_assember.transform(df)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [44]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [45]:
finalized_data = output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [46]:
from pyspark.ml.regression import LinearRegression


# train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])

# train the linearRegression Model
regressor=LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor=regressor.fit(train_data)

In [47]:
test_data

DataFrame[Independent Features: vector, Salary: int]

In [53]:
train_data

DataFrame[Independent Features: vector, Salary: int]

In [48]:
# Coefficients
regressor.coefficients

DenseVector([-263.7076, 1767.624])

In [49]:
#Intercept
regressor.intercept

19919.060052212404

In [50]:
# prediction
pred_result= regressor.evaluate(test_data)

In [51]:
pred_result.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [29.0,4.0]| 20000|19342.03655352618|
+--------------------+------+-----------------+



In [52]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(657.9634464738192, 432915.89689570636)